Pointnet Trial

Importing Libraries

In [39]:
from __future__ import print_function

import gym   # Environment Simulation
import os    # Operation with local space
import random# Random Seed generation
import wandb # Weight and bias logging
import argparse# for setting argument
# Deep Neural Network components
import torch
import torch.nn.parallel
import torch.utils.data
import torch.nn.functional as F
import torch.optim as optim
#from  torchsummary import summary
from torch.utils.data import Dataset
# Arrays and Rotations
import numpy as np
from numpy.random import RandomState
from scipy.spatial.transform import Rotation as R
# Local Imports
import dex_envs # Custom environemnt
#from arguments_pointnet import get_args
from models.pointnet import PointNetClsAndPose

In [40]:
print(PointNetClsAndPose())

PointNetClsAndPose(
  (feat_net): PointNetfeatTwoStream(
    (feat_net): PointNetfeat(
      (conv1): Conv1d(12, 64, kernel_size=(1,), stride=(1,))
      (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (conv3): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (cls_fc1): Linear(in_features=1024, out_features=512, bias=True)
  (cls_fc2): Linear(in_features=512, out_features=256, bias=True)
  (cls_fc3): Linear(in_features=256, out_features=2, bias=True)
  (cls_bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (cls_bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (cls_dropout): Dropout(p=0.3, inplace=False)
  

In [24]:
# always raise numpy error
np.seterr(all='warn')

{'divide': 'warn', 'over': 'warn', 'under': 'warn', 'invalid': 'warn'}

In [25]:
# do not enable wandb output
os.environ["WANDB_SILENT"] = "true"
#os.environ

Here are the param for the training

batch_size = 128

num_points = 128

workers = 128

n_epoch = 30

expID = 0

seed - 125

train_name = []# Name of training environments

test_name = []# Name of test environments

dave_dir = "dex_logs/"

std_data_aug = 0.00384

alpha = 0.459#coefficient for classification and relative rotation estimation

no_safe actio="store_true"

lr = 0.0004086

output_dim = 512 bottleneck dim of Pointnet; claimed to be important by the paper

In [41]:
parser = argparse.ArgumentParser()
parser.add_argument('--batch_size', type=int, default=128, help='input batch size')

_StoreAction(option_strings=['--batch_size'], dest='batch_size', nargs=None, const=None, default=128, type=<class 'int'>, choices=None, help='input batch size', metavar=None)

In [42]:
args = parser.parse_args(args=[])

In [43]:
type(args)

argparse.Namespace

In [63]:
args.num_points = 128
args.workers = 128
args.n_epoch = 30
args.expID = 0
args.seed = 125
#train_name = []# Name of training environments
#test_name = []# Name of test environments
args.save_dir = "dex_logs/"
args.std_data_aug = 0.00384
args.alpha = 0.459#coefficient for classification and relative rotation estimation
args.no_safe="store_true"
args.lr = 0.0004086
args.output_dim = 512# bottleneck dim of Pointnet; claimed to be important by the paper

In [64]:
random.seed(args.seed)

In [65]:
torch.manual_seed(args.seed)

In [66]:
run_name = '{}_{:04d}'.format('pointnet', args.expID)

In [67]:
run_name

'pointnet_0000'

In [68]:
def create_save_dirs(args):
    """
    To create save directory.
    Expects args to have "save_dir argument" key
    agent_type -hard coded to pointnet
    """
    agent_type = 'pointnet'
    # create method directory
    args.save_dir = os.path.join(args.save_dir, agent_type)
    if not os.path.exists(args.save_dir):
        os.makedirs(args.save_dir)
    # check for existing log
    exp_path = os.path.join(
        args.save_dir, '{}_{:04d}'.format(agent_type, args.expID))
    if not os.path.exists(exp_path):
        os.makedirs(exp_path)
    args.save_path = exp_path
    print('*' * 40)
    print('** starting a new run')
    print('*' * 40)
def init_wandb(args, run_name):
    wandb.init(name=run_name, id=run_name, resume=None,
               save_code=True, anonymous="allow")
def log_callback(logged_dict):
    wandb.log(logged_dict)



In [69]:
init_wandb(args, run_name)
create_save_dirs(args)

****************************************
** starting a new run
****************************************


In [ ]:
#train_cls_pose(args)

In [100]:
def get_accuracy(pred_class, target):
    pred_choice = pred_class.data.max(1)[1]
    correct = pred_choice.eq(target.data).cpu().sum()
    accuracy = correct.item() / float(args.batch_size)
    return accuracy


def arcsine_loss(pred_rotation, rotation):
    assert len(pred_rotation.shape) == 2 and pred_rotation.shape[1] == 9
    assert len(rotation.shape) == 2 and rotation.shape[1] == 9
    frobenius = torch.sqrt(torch.sum((pred_rotation - rotation) ** 2, dim=1))
    loss = 2 * \
        torch.arcsin(torch.minimum(torch.ones_like(
            frobenius), frobenius / (2 * np.sqrt(2))))
    return loss.mean(dim=0)


def compute_geodesic_distance_from_two_matrices(m1, m2):
    batch = m1.shape[0]
    m = torch.bmm(m1, m2.transpose(1, 2))  # batch*3*3
    cos = (m[:, 0, 0] + m[:, 1, 1] + m[:, 2, 2] - 1)/2
    cos = torch.min(cos, torch.autograd.Variable(torch.ones(batch)))#.cuda()))
    cos = torch.max(cos, torch.autograd.Variable(torch.ones(batch))*-1)#.cuda())*-1)
    theta = torch.acos(cos)
    return theta

In [101]:
class TwoStreamDataset(Dataset):
    def __init__(self, env_names, num_points=2500, data_aug=True, std_data_aug=0.02, num_data=100000, seed=123):
        self.envs = [makeEnv(env_name, 0, args)() for env_name in env_names]
        self.num_classes = len(env_names)
        for env in self.envs:
            env.reset()
            assert len(env.sim.model.mesh_vertadr) == 13, '{} meshes found, expecting 13 (env: {})'.format(len(env.sim.model.mesh_vertadr), env)
        self.rand = RandomState(seed)
        # self.rand = np.random
        self.num_points = num_points
        self.data_aug = data_aug
        self.num_data = num_data
        self.std_data_aug = std_data_aug

    def _get_points(self, env):
        vert_start_adr = env.sim.model.mesh_vertadr[-1]
        object_vert = env.sim.model.mesh_vert[vert_start_adr:]
        # select some number of object vertices
        selected = self.rand.randint(low=0, high=object_vert.shape[0], size=self.num_points)
        sampled_points = object_vert[selected].copy()
        assert sampled_points.shape[0] == self.num_points and sampled_points.shape[1] == 3
        object_normals = env.sim.model.mesh_normal[vert_start_adr:]
        sampled_normals = object_normals[selected].copy()
        assert sampled_normals.shape[0] == self.num_points and sampled_normals.shape[1] == 3
        return sampled_points, sampled_normals

    def _normalize(self, point_set):
        """zero-center and scale to unit sphere"""
        point_set = point_set - np.expand_dims(np.mean(point_set, axis=0), 0)  # center
        dist = np.max(np.sqrt(np.sum(point_set ** 2, axis=1)), 0)
        point_set = point_set / dist  # scale
        return point_set

    def _augment(self, point_set):
        # random jitter
        point_set += self.rand.normal(0,self.std_data_aug, size=point_set.shape)
        return point_set

    def __getitem__(self, index):
        target = index % self.num_classes
        sampled_points, sampled_normals = self._get_points(
            self.envs[target])
        # apply random rotation to first point set
        first_rotation = R.random()
#        point_set1 = np.matmul(sampled_points, first_rotation.as_dcm().T)
        point_set1 = np.matmul(sampled_points, first_rotation.as_matrix().T)

        # apply random rotation to second point set
        second_rotation = R.random()
#        point_set2 = np.matmul(sampled_points, second_rotation.as_dcm().T)
        point_set2 = np.matmul(sampled_points, second_rotation.as_matrix().T)

        # apply same rotations to normals
#        normal_set1 = np.matmul(sampled_normals, first_rotation.as_dcm().T)
        normal_set1 = np.matmul(sampled_normals, first_rotation.as_matrix().T)

        normal_set2 = np.matmul(
#            sampled_normals, second_rotation.as_dcm().T)
            sampled_normals, second_rotation.as_matrix().T)

        # obtain the rotation between two rotated point sets
        rotation_diff = np.matmul(
#            second_rotation.as_dcm(), first_rotation.inv().as_dcm())
            second_rotation.as_matrix(), first_rotation.inv().as_matrix())

        rotation_diff = rotation_diff.astype(
            np.float32).flatten()  # reformat for training

        # zero-center and scale to unit sphere
        point_set1 = self._normalize(point_set1)
        point_set2 = self._normalize(point_set2)

        # data augmentation
        if self.data_aug:
            point_set1 = self._augment(point_set1)
            point_set2 = self._augment(point_set2)

        return_set1 = np.concatenate(
            [point_set1, normal_set1], axis=-1).astype(np.float32)
        return_set2 = np.concatenate(
            [point_set2, normal_set2], axis=-1).astype(np.float32)

        return return_set1, return_set2, target, rotation_diff

    def __len__(self):
        return int(self.num_data)

In [102]:
#args.train_names = args.train_names if args.train_names else ALL_TRAIN
args.train_names=['knife']

In [103]:
print(args.train_names, args.num_points, args.std_data_aug, args.seed)

['knife'] 128 0.00384 125


In [104]:
def makeEnv(env_name, idx, args):
    """return wrapped gym environment for parallel sample collection (vectorized environments)"""
    def helper():
        e = gym.make('{}-rotate-v1'.format(env_name))
        e.seed(args.seed + idx)
        return e
    return helper

In [105]:
train_dataset = TwoStreamDataset(args.train_names, num_points=args.num_points, data_aug=True,std_data_aug=args.std_data_aug, seed=args.seed)

In [106]:
print(train_dataset.envs[0].sim.model.mesh_vertadr[-1],
      type(train_dataset.envs[0].sim.model.mesh_vert[train_dataset.envs[0].sim.model.mesh_vertadr[-1]:]),
      train_dataset.envs[0].sim.model.mesh_vert[train_dataset.envs[0].sim.model.mesh_vertadr[-1]:].shape)

27203 <class 'numpy.ndarray'> (248, 3)


In [107]:
train_dataloader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=args.batch_size,
        shuffle=True,
        num_workers=int(36),
        pin_memory=True)

In [109]:
model = PointNetClsAndPose(num_classes=len(args.train_names), output_dim=args.output_dim)
#model.cuda()
optimizer = optim.Adam(model.parameters(), lr=args.lr, betas=(0.9, 0.999))
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
num_batch = len(train_dataloader)
#device = torch.device("cuda")
device = torch.device("cpu")
count = 0
print_count=1000
count=0
for epoch in range(args.n_epoch):
    scheduler.step()
    for i, data in enumerate(train_dataloader):
        logged_dict = dict()
        point_set1, point_set2, target, rotation = data
        point_set1 = point_set1.to(device, non_blocking=True)
        point_set2 = point_set2.to(device, non_blocking=True)
        target = target.to(device, non_blocking=True)
        rotation = rotation.to(device, non_blocking=True)
        # conv implementation needs transpose
        point_set1 = point_set1.transpose(2, 1)
        point_set2 = point_set2.transpose(2, 1)
        optimizer.zero_grad()
        pred_class, pred_rotation = model(point_set1, point_set2)
        cls_loss = F.nll_loss(pred_class, target)
        pose_loss = arcsine_loss(pred_rotation, rotation)
        loss = args.alpha * cls_loss + (1 - args.alpha) * pose_loss
        loss.backward()
        optimizer.step()
        # calculate accuracy
        accuracy = get_accuracy(pred_class, target)
        geodesic = compute_geodesic_distance_from_two_matrices(
            pred_rotation.reshape(-1, 3, 3), rotation.reshape(-1, 3, 3)).mean(axis=0)
        
        logged_dict['train_cls_loss'] = cls_loss
        logged_dict['train_pose_loss'] = pose_loss
        logged_dict['train_loss'] = loss
        logged_dict['train_accuracy'] = accuracy
        logged_dict['train_geodesic'] = geodesic
        count+=1
        if count%print_count==0:
            print('[{}: {}/{}] cls loss: {:.3f}, pose loss: {:.3f}, loss: {:.3f}, accuracy: {:.3f}, geodesic: {:.3f}'.format(
                epoch, i, num_batch, cls_loss.item(), pose_loss.item(), loss.item(), accuracy, geodesic))
        log_callback(logged_dict)


[1: 217/782] cls loss: 0.000, pose loss: 0.213, loss: 0.115, accuracy: 1.000, geodesic: 0.322
[2: 435/782] cls loss: 0.000, pose loss: 0.213, loss: 0.115, accuracy: 1.000, geodesic: 0.319
[3: 653/782] cls loss: 0.000, pose loss: 0.203, loss: 0.110, accuracy: 1.000, geodesic: 0.302
[5: 89/782] cls loss: 0.000, pose loss: 0.182, loss: 0.098, accuracy: 1.000, geodesic: 0.290
[6: 307/782] cls loss: 0.000, pose loss: 0.182, loss: 0.099, accuracy: 1.000, geodesic: 0.254
[7: 525/782] cls loss: 0.000, pose loss: 0.188, loss: 0.102, accuracy: 1.000, geodesic: 0.294
[8: 743/782] cls loss: 0.000, pose loss: 0.198, loss: 0.107, accuracy: 1.000, geodesic: 0.289
[10: 179/782] cls loss: 0.000, pose loss: 0.167, loss: 0.090, accuracy: 1.000, geodesic: 0.269
[11: 397/782] cls loss: 0.000, pose loss: 0.170, loss: 0.092, accuracy: 1.000, geodesic: 0.243
[12: 615/782] cls loss: 0.000, pose loss: 0.213, loss: 0.115, accuracy: 1.000, geodesic: 0.338
[14: 51/782] cls loss: 0.000, pose loss: 0.175, loss: 0.09

In [110]:
torch.save(model.state_dict(), '%s/cls_pose_model.pth' %(args.save_path))##save_path added using create save directory function
torch.save(model.feat_net.state_dict(),'%s/feat_model.pth' % (args.save_path))